In [1]:
import chess
import chess.engine
import random
import numpy as np

import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

In [2]:
def random_board(max_depth = 200):
    board = chess.Board()
    depth = random.randrange(0, max_depth)
    for _ in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break
    return board

def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci("./stockfish/stockfish-windows-x86-64-avx2") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result["score"].white().score()
        return score

def square_to_index(square):
    squares_index = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4,'f':5, 'g':6, 'h':7}
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
    board3d = np.zeros((14, 8, 8), dtype = np.int8)
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8,8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8,8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1
    aux = board.turn
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1
        board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    return board3d
    
def build_model(conv_size, conv_depth):
    board3d = layers.Input(shape = (14,8,8))
    x = board3d
    for _ in range(conv_depth):
        x = layers.Conv2D(filters = conv_size, kernel_size = 3, padding = "same", activation = "relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, "relu")(x)
    x = layers.Dense(1, "sigmoid")(x)
    return models.Model(inputs = board3d, outputs = x)
# model = build_model(32, 4)

In [3]:
# for _ in range(3):
#     board = random_board()
#     display(board)
#     print("Board Evaluation: ", stockfish(board, 10))

In [4]:
start = 0
iterations = 100 + start

for iteration in range(start, iterations):
    n = 1000
    large_tensor = np.zeros((n, 14, 8, 8))  
    scores = np.zeros(n)
    for i in range(n):
        ran_board = random_board()
        tensor = split_dims(ran_board)
        large_tensor[i] = tensor
        scores[i] = stockfish(ran_board, 10)
    np.savez(f"BoardStatesAndScores/board_states_and_scores_TROY2_{iteration}.npz", states = large_tensor, scores = scores)
    print(f"{iteration-start+1}/{iterations-start}")

1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
